### Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install lxml html5lib beautifulsoup4
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print ('Libraries imported!')

Solving environment: done

# All requested packages already installed.

Libraries imported!


### 1 GENERATE LIST FROM PARSED URL

 Get the Wikipedia URL from which we shall collect the data and the csv file for the  

In [2]:
# DOWNLOADING CSV FILE FOR TORONTOS POST CODES
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

# RETRIEVING DATA USING THE REQUEST.GET METHOD 
#url='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
trt= requests.get(url)
trt
#trt.text # incase you want to see the text on  the wikipedia page

<Response [200]>

In [3]:
#CREATING THE BEAUTIFUL SOUP OBJECT
soup= BeautifulSoup(trt.text, "html.parser")

In [4]:
#POPULATING THE TABLE
#Firstly create a list
table_contents=[]
#after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        #postal code contains upto 3 characters extract that using tablerow.p.text
        #Next use split ,strip and replace functions for getting Borough and Neighborhood information
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        #Append to the list
        table_contents.append(cell)

# print(table_contents)
#Create a dataframe with list
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df= df.sort_values(by='PostalCode',ascending= True)
df

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
58,M1N,Scarborough,"Birch Cliff, Cliffside West"


###  GET AND APPEND THE LATITUDE AND LONGITUDES OF TORONTOS POSTAL CODES

In [6]:
#EITHER APPEND THE ABOVE TABLE WITH THE CSV FILE THAT WE DOWNLOADED EARLIER
codes=pd.read_csv('Geospatial_Coordinates.csv')
codes
dft = pd.merge(df,codes,left_on='PostalCode',right_on='Postal Code', how='outer')
dft = dft.drop( 'Postal Code', axis=1)
dft

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [7]:
#OR RUN THE WHILE LOOP
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

#loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1] ```

In [8]:
#USING GEOLOCATOR TO GET LATITUDE AND LONGITUDE FOR TORONTO
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [9]:
# create map of Toroonto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(dft['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### 2 EXPLORE TORONTO NEIGHBORHOODS

In [10]:
#FOURSQUARE API CREDENTIALS
CLIENT_ID ='KXWFKHV31RVPPPK1VROSORH4UXLDSVXZWPKTKGQQCC5RQP2Y' # your Foursquare ID
CLIENT_SECRET ='IJ5NCS2FLN4ZMKDSVW2YDX1RORPRIHIF5H4SHAIEUJWREJI3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT= 100 # A default Foursquare API limit value
TOKEN='LCX5WFKJUQRLTVHGTAKL5Y01HLGJ3RXS0K1HC0LLCVJBV01J'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KXWFKHV31RVPPPK1VROSORH4UXLDSVXZWPKTKGQQCC5RQP2Y
CLIENT_SECRET:IJ5NCS2FLN4ZMKDSVW2YDX1RORPRIHIF5H4SHAIEUJWREJI3


In [11]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)                             

In [12]:
 # FILTER ONLY BOUGHTS CONTAINING TORONTO
dftrt = dft[dft['Borough'].str.contains("Toronto")].reset_index(drop=True)

dftrt

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923
5,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197
7,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
8,M4S,Central Toronto,Davisville,43.704324,-79.388790
9,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [13]:
trt_venues = getVenues(names=dftrt['Neighborhood'],
                                   latitudes=dftrt['Latitude'],
                                   longitudes=dftrt['Longitude']
                                  )
# this returns a list of all neiborhhoods

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [14]:
trt_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park


In [15]:
#LETS SEE VENUES IN EACH NIGHBOURHOOD
#reset index so neighborhood column gets to be considered 
trt_venues.groupby('Neighborhood').count().reset_index()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,59,59,59,59,59,59
1,"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
2,"CN Tower, King and Spadina, Railway Lands, Har...",16,16,16,16,16,16
3,Central Bay Street,64,64,64,64,64,64
4,Christie,16,16,16,16,16,16
5,Church and Wellesley,70,70,70,70,70,70
6,"Commerce Court, Victoria Hotel",100,100,100,100,100,100
7,Davisville,32,32,32,32,32,32
8,Davisville North,10,10,10,10,10,10
9,"Dufferin, Dovercourt Village",16,16,16,16,16,16


In [16]:
# UNIQUE VALUES CURATED FROM RETURNED VENUES
print('There are {} uniques categories.'.format(len(trt_venues['Venue Category'].unique())))

There are 232 uniques categories.


### 3 ANALYZE TORONTO NEIGHBORHOODS

In [17]:
# ONE HOT ENCODING
trt_onehot = pd.get_dummies(trt_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
#ngh=list(trt_venues['Neighborhood'])
trt_onehot['Neighborhood'] = trt_venues['Neighborhood']
#trt_onehot.set_index('Neighborhood')

# MAKE NEIGHBORHOOOD THE FIRST COLUMN
trt_onehot= trt_onehot[['Neighborhood'] + [ col for col in trt_onehot.columns if col != 'Neighborhood' ] ]
trt_onehot.head(3)

# move neighborhood column to the first column and all List is nudged forward
#fixed_columns = [trt_onehot.columns[-1]] + list(trt_onehot.columns[:-1])
#trt_onehot = trt_onehot[fixed_columns]
#trt_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
trt_onehot.shape

(1571, 232)

Grouping rows by "Neighborhood" and taking their "Mean" of Occurence 

In [19]:
trt_grped = trt_onehot.groupby('Neighborhood').mean().reset_index()
trt_grped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.016949,0.050847,0.000000,0.000000,0.00,0.016949,0.016949,0.000000,0.033898,0.000000,0.016949,0.016949,0.0000,0.000000,0.0000,0.00000,0.016949,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.016949,0.00,0.0000,0.000000,0.033898,0.000000,0.000000,0.000000,0.000000,0.016949,0.050847,0.067797,0.000000,0.000000,0.00,0.00000,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.00000

In [20]:
trt_grped.shape

(38, 232)

Printing each neighborhood along with the top 5 most common venues

In [21]:
top_venues = 5

for hood in trt_grped['Neighborhood']:
    print("----"+hood+"----")
    temp = trt_grped[trt_grped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2          Bakery  0.05
3      Restaurant  0.03
4  Farmers Market  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3   Convenience Store  0.05
4  Italian Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.05
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Coffee Shop  0.06
4    Cand

Defining Function to Sort Venues in Descending Order

In [22]:
def common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]
# top_venues is already specified in previous cell

Putting the above list in a dataframe

In [23]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
hood_venues_sorted = pd.DataFrame(columns=columns)
hood_venues_sorted['Neighborhood'] = trt_grped['Neighborhood']
# use the define function above 
for ind in np.arange(trt_grped.shape[0]):
    hood_venues_sorted.iloc[ind, 1:] = common_venues(trt_grped.iloc[ind, :],top_venues)

hood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Seafood Restaurant,Farmers Market,Juice Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Bar,Intersection,Bakery,Restaurant,Climbing Gym,Furniture / Home Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Salad Place,Burger Joint,Thai Restaurant,Japanese Restaurant,Department Store
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports,Restaurant,Coffee Shop


### CLUSTERING TORONTO NEIGHBORHOODS

In [24]:
# set number of clusters
kclusters = 6
# lets drop the neighborhood column for once. we are using the mean occurrence dataframe
trt_grped_cluster = trt_grped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,init='k-means++', random_state=1).fit(trt_grped_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Adding Cluster Labels to Dataframe

In [25]:
# add clustering labels
# using the INSERT function to make cluster labels first column or convert array to datframe and add
#hood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
hood_venues_sorted['Cluster Labels']= pd.DataFrame(kmeans.labels_)
#hood_venues_sorted= hood_venues_sorted['Cluster Labels'] + [ col for col in hood_venues_sorted.columns if col != 'Cluster Labels'] 

trt_merge = dftrt

# merge using JOIN hood_venue_sorted in preference with dftrt to add latitude/longitude for each neighborhood
#reset index to avoid issues
trt_merge = trt_merge.join(hood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
trt_merge.head(3) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,Trail,Pub,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,4.0
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,Park,Convenience Store,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,5.0
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Furniture / Home Store,Ice Cream Shop,Caribbean Restaurant,Pub,Café,0.0


In [26]:
print('number of NAN values is:', trt_merge['Cluster Labels'].isnull().sum())
print(trt_merge.shape)

number of NAN values is: 1
(39, 16)


In [27]:
#drop the one value that has no values
trt_merge=trt_merge.dropna(subset=['Cluster Labels'])
print(trt_merge.shape)

(38, 16)


In [28]:
trt_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,Trail,Pub,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,4.0
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,Park,Convenience Store,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,5.0
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Furniture / Home Store,Ice Cream Shop,Caribbean Restaurant,Pub,Café,0.0
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,Sandwich Place,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Pub,Light Rail Station,Fish & Chips Shop,Italian Restaurant,Steakhouse,Restaurant,0.0
4,M4M,East Toronto,Studio District,43.659526,-79.340923,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Seafood Restaurant,Cheese Shop,0.0


In [29]:
# create map
# kcluster were defined when calculating kmeans 
clustermap = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trt_merge['Latitude'], trt_merge['Longitude'], trt_merge['Neighborhood'], trt_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(clustermap)
       
clustermap
 # if ones gets an error on cluster-1 simply convert it to an integer by int(cluster)-1

### EXAMINE TORONTO NEIGHBORHOODS

Cluster 1 Examination

In [30]:
trt_merge.loc[trt_merge['Cluster Labels'] == 0, trt_merge.columns[[1] + list(range(5, trt_merge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Furniture / Home Store,Ice Cream Shop,Caribbean Restaurant,Pub,Café,0.0
3,East Toronto,Sandwich Place,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Pub,Light Rail Station,Fish & Chips Shop,Italian Restaurant,Steakhouse,Restaurant,0.0
4,East Toronto,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Seafood Restaurant,Cheese Shop,0.0
6,Central Toronto,Gym,Dance Studio,Sandwich Place,Department Store,Hotel,Food & Drink Shop,Breakfast Spot,Convenience Store,Park,Gym / Fitness Center,0.0
7,Central Toronto,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Gym / Fitness Center,Ice Cream Shop,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,0.0
8,Central Toronto,Sandwich Place,Dessert Shop,Coffee Shop,Café,Italian Restaurant,Pizza Place,Sushi Restaurant,Gym,Gourmet Shop,Toy / Game Store,0.0
10,Central Toronto,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Supermarket,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Light Rail Station,0.0
12,Downtown Toronto,Bakery,Coffee Shop,Pizza Place,Italian Restaurant,Pub,Café,Restaurant,Bank,Diner,Indian Restaurant,0.0
13,Downtown Toronto,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Fast Food Restaurant,Pub,Hotel,Mediterranean Restaurant,0.0
14,Downtown Toronto,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Dessert Shop,Shoe Store,0.0


Cluster 2 Examination

In [31]:
trt_merge.loc[trt_merge['Cluster Labels'] == 1, trt_merge.columns[[1] + list(range(5, trt_merge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
5,Central Toronto,Park,Swim School,Bus Line,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,1.0


Cluster 3 Examination

In [32]:
trt_merge.loc[trt_merge['Cluster Labels'] == 2, trt_merge.columns[[1] + list(range(5, trt_merge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
11,Downtown Toronto,Park,Playground,Trail,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,2.0
24,Central Toronto,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,2.0


Cluster 4 Examination

In [33]:
trt_merge.loc[trt_merge['Cluster Labels'] == 3, trt_merge.columns[[1] + list(range(5, trt_merge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
23,Central Toronto,Pool,Home Service,Garden,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,3.0


Cluster 5 Examination

In [34]:
trt_merge.loc[trt_merge['Cluster Labels'] == 4, trt_merge.columns[[1] + list(range(5, trt_merge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,East Toronto,Health Food Store,Trail,Pub,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,4.0


Cluster 6 Examination

In [34]:
trt_merge.loc[trt_merge['Cluster Labels'] == 5, trt_merge.columns[[1] + list(range(5, trt_merge.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1,East York/East Toronto,Park,Convenience Store,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,5.0


The end